## Authoritarian Framework

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph, MessageGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [30]:
unique_id = "Authoritarian Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [31]:
# from langsmith import Client

# client = Client()

In [65]:
class PromptReview(BaseModel):
    """Review of the prompt"""
    prompt: str = Field(description="The most recent prompt")
    feedback: str = Field(description="Feedback on the most recent prompt")

class ApprovalDecision(BaseModel):
    """Decision on the approval of the prompt"""
    approved: bool = Field(description="Whether the prompt was approved or not")
    explanation: str = Field(description="A detailed explanation of why the prompt was approved or not")

class AdvisorPrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class AdvisorAgent:
    """
    Advisor Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: AdvisorPrinciples, temp: float = 0.5, model: str = "gpt-4o"):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = SystemMessage(content=f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}
You must use your expertise and core principles to guide all your thinking. You must speak only as an expert in your field.""") 
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def review_prompt(self, prompt: str, additional_info: str) -> PromptReview:
        """
        Generates a review of the prompt.
        """
        template = """Your task is to think outside the box and provide feedback on the prompt below with creative recommendations on how to improve it in light of your core principles:
{prompt}

Your feedback must be less than 100 words so think carefully about the most critical aspects of the prompt that need improvement.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow when providing feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN IN PLACE.
You will be penalized if you do not follow these guidelines.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}. 
2. Identify the most critical aspects of the prompt that need improvement. 
3. Think outside the box to provide creative feedback with recommendations on how to improve the prompt in light of your core principles.
4. Ensure that your feedback is less than 100 words.
5. Ensure that your feedback follows the strict guidelines provided above.
6. Submit your feedback.

Return only the original prompt and your feedback in JSON fromat below:

{{
    "prompt": "Most recent prompt",
    "feedback": "Feedback and recommendations for the original prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptReview)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.prompt and completion.feedback:
                    return completion
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")
        
    def approval(self, prompt: str, additional_info: str):
        """
        Agent to approve or reject the prompt.
        """
        template = """Your task is to review the prompt below and decide whether or not it should be approved in light of your core principles:
{prompt}

Return the boolean value "True" if you approve of the prompt and "False" otherwise.
You must also provide a detailed explanation of why you approved or rejected the prompt.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST NOT contradict when explaining your decision:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN IN PLACE.
You will be penalized if you do not follow these guidelines.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}.
2. Determine whether the prompt will be effective in instructing the model to perform the desired task.
3. If you believe the prompt is effective, approve it for use by the model.
4. Explain why you approved or rejected the prompt in light of your core principles.
5. Submit your decision.

Return only the approval decision and your explanation in JSON format below:

{{
    "approved": "True/False",
    "explanation": "Detailed explanation of why the prompt was approved or rejected"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly.
"""
        pydantic_parser = PydanticOutputParser(pydantic_object=ApprovalDecision)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.approved is not None and completion.explanation:
                    return completion.approved
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")


In [66]:
class Advisors(BaseModel):
    """Details of advisors generated by the leader agent."""
    positions: List[str] = Field(description="Positions of the advisors")
    core_principles: List[List[str]] = Field(description="Core principles of the advisors")

class RouteDecision(BaseModel):
    """Decision on the next advisor to process."""
    next: str = Field(description="The next advisor to process")

class UpdatedPrompt(BaseModel):
    """Updated prompt based on feedback from the advisor agent."""
    updated_prompt: str = Field(description="Updated prompt based on feedback from the advisor agent")

class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates with advisor agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, additional_info: str = None, temp: float = 0.0, model: str = "gpt-4o", advisors: List[AdvisorAgent] = None):
        self.base_prompt = base_prompt
        self.additional_info = additional_info
        self.system_message = SystemMessage(content=f"""You are an experienced senior AI professional. You specialise in prompt engineering. 
You have in-depth knowledge of large language models and prompt engineering best practices. Use this knowledge to inform all your decisions.""")
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.advisors = advisors
        self.iterations = 0
    
    def run_approval(self, prompt: str) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(advisor.approval, prompt, self.additional_info) for advisor in self.advisors]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: dict) -> RouteDecision:
        """
        LeaderAgent to decide the next advisor or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state["prompt"])
        print("Approval results:", approval_results)
        if all(approval_results) or (self.iterations > 10):
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
        else:
            # Only ask for advise from advisors that disapproved the prompt
            disapproved_advisors = [advisor for i, advisor in enumerate(self.advisors) if not approval_results[i]]
            disapproved_advisors_details = {advisor.position: advisor.core_principles for advisor in self.advisors}
            options = [advisor.position for advisor in disapproved_advisors]
            # shuffle the options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + positions
            template = """Your task is to review the prompt below and decide the next advisor to provide feedback:
{prompt}

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

The details of all advisors are their core principles are as follows: 
{advisor_roles}

Select one of the below advisors that disapproved the prompt to provide feedback on how to improve it: 
{options}

Think carefully about which aspects of the prompt need improvement and which advisor would be best suited to provide feedback on those aspects.
If you think multiple aspects of the prompt need improvement, select the most suitable advisor to provide feedback on the most critical aspect of the prompt.

Return only the next advisor in JSON format below:

{{
    "next": "Next advisor",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
            pydantic_parser = PydanticOutputParser(pydantic_object=RouteDecision)
            prompt_template = PromptTemplate(
                system_message=self.system_message,
                template=template,
                input_variables=["prompt", "additional_info", "advisor_roles", "options"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
            )
            chain = prompt_template | self.llm | pydantic_parser
            for _ in range(3):
                try:
                    output = chain.invoke({"prompt": state["prompt"], "additional_info": self.additional_info, "advisor_roles": str(disapproved_advisors_details), "options": str(options)})
                    break
                except Exception as e:
                    print("Exception occurred:", e)
                    continue
            # self.updates += 1
            # if output.next == "FINISH" or self.updates == 5:
            #     return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
            # else:
                # return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}
            return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}

    def update_prompt(self, prompt: str, feedback: str, history) -> str:
        """
        Updates the prompt with the feedback from the advisor agent.
        """
        template = """Your task is to make updates to the prompt:
{prompt}

You must consider the feedback provided:
{feedback} 

You must also consider the discussion history below prior to making changes to the prompt to ensure you do not repeat any mistakes: 
{history}

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN IN PLACE.
You will be penalized if you do not follow these guidelines.

Your update process should be as follows:
1. Read the prompt and feedback carefully.
2. Review the discussion history to ensure you do not repeat any mistakes.
3. Implement the feedback provided.
4. Ensure that your updated prompt instructs the model as expected.
5. Ensure that your update follows the strict guidelines provided above.
6. Submit your updated prompt.

Return only the updated prompt in JSON format:

{{
    "prompt": "Updated prompt",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "feedback", "history", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": prompt, "feedback": feedback, "history": history,  "additional_info": self.additional_info})
                if output.updated_prompt:
                    return output.updated_prompt
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", output)
            except Exception as e:
                print("Exception occurred:", e)
                continue

    def construct_advisor_graph(self):
        """
        Constructs a graph of advisor agents based on their roles and functions.
        """

        def agent_node(state, agent, position):
            try:
                result = agent.review_prompt(state["prompt"], self.additional_info)
                print(f"Result: {result}")
                feedback = result.feedback
                updated_prompt = self.update_prompt(state["prompt"], feedback, state["messages"])
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Feedback: {feedback}", name=position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {position}: {e}")
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Error: Parsing failed for {position} - {e}", name=position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": state["prompt"],
                    "next": "leader"
                }
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Sequence[BaseMessage]
            next: str
            prompt: str

        workflow = StateGraph(AgentState)
        for advisor in self.advisors:
            # Create a node for each advisor agent
            node = functools.partial(agent_node, agent=advisor, position=advisor.position)
            workflow.add_node(advisor.position, node)
        workflow.add_node("leader", self.leader_decision)

        members = [advisor.position for advisor in self.advisors]
        for member in members:
            # We want our advisors to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("leader", lambda x: x["next"], conditional_map)
        workflow.set_entry_point("leader")
        graph = workflow.compile()

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of advisor agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"{self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "leader"
        }

        # Construct the graph
        graph = self.construct_advisor_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        # Run the graph
        for s in graph.stream(
            initial_state,
            {"recursion_limit": 50}
            ):
            if "__end__" not in s:
                # print most recent message in state
                print(s)
                print("----")
                continue
        
        # if not os.path.exists("prompt_history_authoritarian.json"):
        #     with open("prompt_history_authoritarian.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_authoritarian.json", "r") as f:
        #     data = json.load(f)
        #     data.append(s)
            
        # with open("prompt_history_authoritarian.json", "w") as f:
        #     json.dump(data, f, indent=4)

        return s


In [67]:
# Prompt design team members
conciseness_and_clarity_principles = AdvisorPrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language to communicate ideas",
    "Always format and structure prompts for easy readability",
])
conciseness_and_clarity_expert = AdvisorAgent("Conciseness and Clarity Expert", conciseness_and_clarity_principles)

contextual_relevance_principles = AdvisorPrinciples([
    "Always provide context to help the model understand the task",
    "Always consider the context in which prompts will be used",
])
contextual_relevance_expert = AdvisorAgent("Contextual Relevance Expert", contextual_relevance_principles)

task_alignment_principles = AdvisorPrinciples([
    "Always ensure that prompts align with the task requirements",
    "Always tailor instructions to the task to guide the model",
    "Always consider the expected output of the model",
])
task_alignment_expert = AdvisorAgent("Task Alignment Expert", task_alignment_principles)

example_demonstration_principal = AdvisorPrinciples([
    "Always provide examples to help the model understand the task",
    "Always ensure examples are relevant and clear",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = AdvisorAgent("Example Demonstration Expert", example_demonstration_principal)

avoiding_bias_principles = AdvisorPrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = AdvisorAgent("Avoiding Bias Expert", avoiding_bias_principles)

incremental_prompting_principles = AdvisorPrinciples([
    "Always provide clear step-by-step instructions to guide the model",
    "Always consider the complexity of the task when providing incremental instructions",
])
incremental_prompting_expert = AdvisorAgent("Incremental Prompting Expert", incremental_prompting_principles)

programming_logic_principles = AdvisorPrinciples([
    "Always ensure that prompts are logically structured, similar to programming logic",
    "Always consider the logical flow of instructions in prompts",
    "Always consider the usefulness of programming logic to the task",
])
programming_logic_expert = AdvisorAgent("Programming Logic Expert", programming_logic_principles)

In [68]:
# Domain team members
mathematics_principles = AdvisorPrinciples([
    "Always think with a mathematical mindset",
    "Always use mathetical operators correctly",
    "Always adhere to mathematical rules",
])
mathematician = AdvisorAgent("Mathematician", mathematics_principles)

word_problem_solving_principles = AdvisorPrinciples([
    "Always pay attention to the keywords in the problem",
    "Always approach problems systematically",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = AdvisorAgent("Word Problem Solver", word_problem_solving_principles)


In [69]:
base_prompt = "{content}.\nPlease output your answer at the end as ##<your answer (arabic numerals)>."
additional_info = "- Solve the math word problem.\n- Output the answer at the end as ##<your answer (arabic numerals)> with no spaces or units."

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
    advisors=[
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        avoiding_bias_expert,
        incremental_prompting_expert,
        programming_logic_expert,
        mathematician,
        word_problem_solver,
    ]
)
for advisor in leader_agent.advisors:
    print("Position: ", advisor.position + "\nCore Principles:\n", advisor.core_principles)


Position:  Conciseness and Clarity Expert
Core Principles:
 - Always write clear and concise prompts
- Always use simple and direct language to communicate ideas
- Always format and structure prompts for easy readability
Position:  Contextual Relevance Expert
Core Principles:
 - Always provide context to help the model understand the task
- Always consider the context in which prompts will be used
Position:  Task Alignment Expert
Core Principles:
 - Always ensure that prompts align with the task requirements
- Always tailor instructions to the task to guide the model
- Always consider the expected output of the model
Position:  Example Demonstration Expert
Core Principles:
 - Always provide examples to help the model understand the task
- Always ensure examples are relevant and clear
- Always demonstrate the expected output of the model
Position:  Avoiding Bias Expert
Core Principles:
 - Always avoid bias in prompts
- Always consider the ethical implications of prompts
Position:  Incre

In [70]:
result = leader_agent.optimise_prompt()

Approval results: [True, False, True, True, True, True, True, True, False]
{'leader': {'messages': [HumanMessage(content='{content}.\nPlease output your answer at the end as ##<your answer (arabic numerals)>.', name='User')], 'next': 'Word Problem Solver', 'prompt': '{content}.\nPlease output your answer at the end as ##<your answer (arabic numerals)>.'}}
----
Result: prompt='Your task is to think outside the box and provide feedback on the prompt below with creative recommendations on how to improve it in light of your core principles: {content}. Please output your answer at the end as ##<your answer (arabic numerals)>. Your feedback must be less than 100 words so think carefully about the most critical aspects of the prompt that need improvement. Below are details of what the prompt is expected to instruct the model to do: - Solve the math word problem. - Output the answer at the end as ##<your answer (arabic numerals)> with no spaces or units. Below are strict guidelines that you MU

In [71]:
result['leader']['prompt']

'Solve the math word problem in {content} and output the final answer at the end as ##<your answer (arabic numerals)> with no spaces or units. For example, if the answer is 42, output ##42. Ensure your calculations are correct before outputting the final answer.'

### Concurrent Runs

In [ ]:
base_prompt = "Classify the grammar in the text as correct or incorrect: {content}"
additional_info = "The prompt should instruct the LLM to output either 'correct' or 'incorrect' based on the grammar in the text."

leader_agent_1 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for advisor in leader_agent_1.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_2 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for advisor in leader_agent_2.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_3 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for advisor in leader_agent_3.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Position:  Sentiment Analysis Specialist
Role:  Expert in identifying and classifying sentiments in text
Function:  Analyze and categorize the sentiment of sentences to improve data annotation quality

Position:  NLP Prompt Engineer
Role:  Developer of optimized and precise prompts for natural language processing models
Function:  Design and iterate on prompt structures that enhance the performance and generalizability of the language model for sentiment classification

Position:  Linguistic Quality Assurance Expert
Role:  Verifier of linguistic accuracy and nuance in sentiment classification
Function:  Review and validate the results of sentiment classification to ensure they align with linguistic nuances and human intuition, providing feedback for further optimization

Position:  Sentiment Analysis Expert
Role:  Responsible for providing domain-specific insights on sentiment classification.
Function:  Analyze a wide range of sentences to determine nuanced sentiment indicators and pro

In [ ]:
# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_advisors=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine good prompts to create great prompts!
You have in-depth knowledge of large language models and prompt engineering best practices. Use knowledge at all times to guide your thinking."""

    template = """Your task is to merge the best parts of the prompts below to create a better prompt.
Carefully consider the strengths of each prompt and how they can be combined effectively whilst maintaining clarity and relevance.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
Consider aspects of good prompts beyond those listed above.
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.

Prompts: {results}

Return only the next advisor to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': [{'feedback': 'Include criteria for classification to ensure consistent interpretation and improve clarity.', 'updated_prompt': 'Classify the sentence as positive or negative: {content}. A positive classification indicates that the sentence expresses a favorable, approving, or optimistic sentiment. A negative classification indicates that the sentence expresses an unfavorable, disapproving, or pessimistic sentiment.'}, {'feedback': 'Clarify if neutral sentiments should be ignored or included. Ensure it explicitly states the treatment of ambiguous cases.', 'updated_prompt': 'Classify the sentence as positive or negative: {content}. A positive classification indicates that the sentence expresses a favorable, approving, or optimistic sentiment. A negative classification indicates that the sentence expresses an unfavorable, disapproving, or pessimistic sentiment. Neutral sentiments should be ign

In [ ]:
print(final_result)

Classify the sentence as positive or negative based on sentiment analysis: {content}. A positive classification indicates that the sentence expresses a favorable, approving, or optimistic sentiment. A negative classification indicates that the sentence expresses an unfavorable, disapproving, or pessimistic sentiment. Neutral sentiments, which show no strong positive or negative tones, should be ignored. For mixed sentiments, assess the overall tone and classify according to the predominant sentiment. In ambiguous cases, where the sentiment may not be clear, use contextual cues and inferred intent to classify as either positive or negative.
